## RNN - MNIST

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('../data/mnist/data/', one_hot=True)

Extracting ../data/mnist/data/train-images-idx3-ubyte.gz
Extracting ../data/mnist/data/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/data/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/data/t10k-labels-idx1-ubyte.gz


In [26]:
tf.reset_default_graph()

####################
# Hyper Parameters #
####################
learning_rate = 0.001
total_epoch = 5
batch_size = 128

# RNN은 Sequential data를 다루는 모델이므로,
# 한 번에 입력받는 개수와 총 몇 단계(time_step)로 이루어져 있는 데이터를 받을지 설정해야한다.
# MNIST에서는 가로 픽셀수(28)를 n_input으로, 세로 픽셀수를 입력 단계(time_step)인 n_step으로 설정해준다.
n_input = 28
n_step = 28
n_hidden =128
n_class = 10


################
# RNN 모델 구성 #
###############
inputs = tf.placeholder(tf.float32, [None, n_step, n_input])
labels = tf.placeholder(tf.float32, [None, n_class])

weights = tf.Variable(tf.truncated_normal([n_hidden, n_class], stddev=0.1))
bias = tf.Variable(tf.truncated_normal([n_class]))

# RNN의 학습에 사용할 Cell을 생성
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

# RNN 생헝
outputs, states = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
outputs = outputs[:, -1, :]
logits = tf.matmul(outputs, weights) + bias

# loss & optimizer
cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# accuracy
correct_mask = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [27]:
################
# RNN 모델 학습 #
################
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(mnist.train.num_examples/batch_size)
    
    for epoch in range(1, total_epoch+1):
        total_cost = 0
        
        for step in range(1, total_batch+1):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # input shape에 맞게 reshape 해준다.
            batch_x = batch_x.reshape([batch_size, n_step, n_input])
            
            loss, _ = sess.run([cross_entropy, train_op],
                               feed_dict={inputs: batch_x, labels: batch_y})
            total_cost += loss
            
        train_x, train_y = mnist.train.images, mnist.train.labels
        train_x = train_x.reshape([-1, n_step, n_input])
        train_acc = sess.run(accuracy, feed_dict={inputs: train_x, labels: train_y})
        print('Epoch: {:04d}'.format(epoch),
              'Avg. cost = {:.5f}'.format(total_cost/total_batch),
              'Train Acc = {:.5f}'.format(train_acc))
        
    print('최적화 완료!')
    
    # Test
    test_x = mnist.test.images.reshape([-1, n_step, n_input])
    test_y = mnist.test.labels
    
    test_acc = sess.run(accuracy, feed_dict={inputs: test_x, 
                                             labels: test_y})
      

print('Test Acc = {:.5f}'.format(test_acc))

Epoch: 0001 Avg. cost = 0.63861 Train Acc = 0.90767
Epoch: 0002 Avg. cost = 0.25816 Train Acc = 0.93642
Epoch: 0003 Avg. cost = 0.18625 Train Acc = 0.95765
Epoch: 0004 Avg. cost = 0.15008 Train Acc = 0.96253
Epoch: 0005 Avg. cost = 0.13548 Train Acc = 0.96704
최적화 완료!
Test Acc = 0.96620
